In [27]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from collections import Counter

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from pymongo import MongoClient
from bson.objectid import ObjectId
from Milestone41 import CRUD


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
try:
    USER = 'aacuser'  
    PASS = 'password'  
    HOST = 'nv-desktop-services.apporto.com'
    PORT = 31802
    DB = 'AAC'
    client = MongoClient(f'mongodb://{USER}:{PASS}@{HOST}:{PORT}/{DB}?authSource=admin')
    print("Connected successfully.")
except Exception as e:
    print(f"Could not connect to MongoDB: {e}")

# Connect to database via CRUD Module
db = CRUD()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
##print(len(df.to_dict(orient='records')))
##print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Jarrett Henkel')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    # FIXME Add in Grazioso Salvare’s logo
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Jarrett Henkel CS-340 MongoDB Authentication'))),
    html.Hr(),
    
    # FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div(
        dcc.Dropdown(
            id='filter-dropdown',
            options=[
                {'label': 'Show All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
            ],
            value='all'
        )
    ),
    # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        row_selectable='single'
    ),
    html.Br(),
    html.Hr(),
    html.Div([
        html.Div(
            id='map-id',
            className='six columns', 
            style={'width': '50%', 'display': 'inline-block'} 
        ),
        html.Div(
            id='graph-id',
            className='six columns',
            style={'width': '50%', 'display': 'inline-block'}
        ),
    ], className='row'),  
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-dropdown', 'value')]
)

def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    query = {}  
    
    if filter_type != 'all':
        if filter_type == 'Water Rescue':
            query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        elif filter_type == 'Mountain or Wilderness Rescue':
            query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        elif filter_type == 'Disaster Rescue or Individual Tracking':
            query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    
    # Fetch and return the filtered data from MongoDB
    filtered_df = pd.DataFrame.from_records(db.read(query))
    filtered_df.drop(columns=['_id'], inplace=True)
    
    return filtered_df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return []
    dff = pd.DataFrame.from_records(viewData)
    breed_count = Counter(dff['breed'])
    total_animals = sum(breed_count.values())
    filtered_breed_count = {}
    other_count = 0
    for breed, count in breed_count.items():
        percentage = (count / total_animals) * 100
        if percentage >= 1:
            filtered_breed_count[breed] = count
        else:
            other_count += count
    if other_count > 0:
        filtered_breed_count["Other"] = other_count
    df_filtered = pd.DataFrame(list(filtered_breed_count.items()), columns=['breed', 'count'])
            
            
    pie_chart = dcc.Graph(        
        figure=px.pie(df_filtered, names='breed', values='count', title='Preferred Animals')
    )
    return [pie_chart]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None or index is None:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10)
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    row = index[0] if index else 0
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connected successfully.
Dash app running on http://127.0.0.1:21129/
